In [121]:
import pandas as pd
import examen
import examen.cargar_datos as cd
import examen.validar_datos as vd
import examen.perfil_datos as ped
import examen.analiticas as an
from IPython.display import display

In [122]:
import importlib
importlib.reload(cd)
importlib.reload(vd)
importlib.reload(ped)
importlib.reload(an)

<module 'examen.analiticas' from '/Users/davidmc/Documents/09-Aprendizaje/01-Diplomados/01-Ciencia_de_Datos/01-Modulo/00-Examen/examen/analiticas.py'>

## Extracción

Se crearon dos módulos dentro del paquete `examen`:
- `cargar_datos.py`: carga de tablas desde el archivo .zip ubicado en `data/` (sin extraer a disco).
- `validar_datos.py`: validación simple comparando pares CSV vs XLSX.

Tras generar reportes completos de diferencias (NaN-safe) para los pares disponibles, se decidió
utilizar los archivos CSV como fuente principal para uniones (joins) y pasos posteriores,
manteniendo los XLSX como referencia.

In [123]:
dfs = cd.cargar_dfs_desde_zip_en_data()
list(dfs.keys())

Leyendo datos/parking.csv -> key=parking_csv
Leyendo datos/usercuisine.csv -> key=usercuisine_csv
Leyendo datos/restaurants.xlsx -> key=restaurants_xlsx
Leyendo datos/users.xlsx -> key=users_xlsx
Leyendo datos/restaurants.csv -> key=restaurants_csv
Leyendo datos/cuisine.csv -> key=cuisine_csv
Leyendo datos/users.csv -> key=users_csv
Leyendo datos/ratings.csv -> key=ratings_csv
Leyendo datos/hours.csv -> key=hours_csv
Leyendo datos/userpayment.csv -> key=userpayment_csv
Leyendo datos/payment_methods.csv -> key=payment_methods_csv


['parking_csv',
 'usercuisine_csv',
 'restaurants_xlsx',
 'users_xlsx',
 'restaurants_csv',
 'cuisine_csv',
 'users_csv',
 'ratings_csv',
 'hours_csv',
 'userpayment_csv',
 'payment_methods_csv']

In [124]:
reps = vd.reporte_todos_pares_completo(dfs)

for base, rep in reps.items():
    print(rep["summary"])
    display(rep["diffs"])

{'base': 'restaurants', 'csv_shape': (130, 21), 'xlsx_shape': (130, 21), 'same_columns': True, 'equals_pandas': False, 'mismatch_cells': 107, 'note': ''}


,row,col,csv,xlsx
0,1,zip,78280,78280
1,2,latitude,22.149709,22.149709
2,2,longitude,-100.976093,-100.976093
3,2,zip,78000,78000
4,3,city,victoria,victoria
...,...,...,...,...
102,126,latitude,22.149192,22.149192
103,126,zip,78220,78220
104,128,latitude,18.875011,18.875011
105,128,longitude,-99.159422,-99.159422


{'base': 'users', 'csv_shape': (138, 19), 'xlsx_shape': (138, 19), 'same_columns': True, 'equals_pandas': False, 'mismatch_cells': 173, 'note': ''}


,row,col,csv,xlsx
0,0,smoker,false,False
1,1,smoker,false,False
2,2,smoker,false,False
3,3,longitude,-99.183,-99.183
4,3,smoker,false,False
...,...,...,...,...
168,134,smoker,false,False
169,135,smoker,true,True
170,136,longitude,-100.944623,-100.944623
171,136,smoker,false,False


In [125]:
dfs_csv = cd.obtener_csvs(dfs)

## Perfilado de DataFrames (CSV)

En esta sección se realiza el **perfilado exploratorio** de todos los DataFrames provenientes de archivos **CSV**, con el objetivo de:
- identificar estructura (filas/columnas),
- revisar distribución y tipo de variables,
- detectar valores faltantes y posibles inconsistencias,
- y documentar la calidad inicial de los datos antes de la integración/modelado.

In [126]:
dfs_perfil = ped.perfilar_todos_los_dfs(dfs_csv)

for name, perfil in dfs_perfil.items():
    print(f"\n=== PERFIL: {name} ===")
    display(perfil)


=== PERFIL: users_csv ===


,Variable,Tipo_inferido,Dtype,Nulos,%Nulos,Unicos,%Cardinalidad,Min,Max,Media,Moda,Top_3
0,userID,Texto,object,0,0.0,138,100.00,NaN,NaN,NaN,U1001,"U1001 (1), U1095 (1), U1089 (1)"
1,latitude,Numerica,float64,0,0.0,128,92.75,18.813348,23.771030,21.810389,None,None
2,longitude,Numerica,float64,0,0.0,126,91.30,-101.054680,-99.067106,-100.291857,None,None
3,weight,Numerica,int64,0,0.0,49,35.51,40.000000,120.000000,64.869565,None,None
4,height,Numerica,float64,0,0.0,38,27.54,1.200000,2.000000,1.667536,None,None
5,birth_year,Numerica,int64,0,0.0,21,15.22,1930.000000,1994.000000,1984.702899,None,None
6,color,Categorica,object,0,0.0,8,5.80,NaN,NaN,NaN,blue,"blue (45), black (21), green (19)"
7,dress_preference,Categorica,object,0,0.0,5,3.62,NaN,NaN,NaN,no preference,"no preference (53), formal (41), informal (35)"
8,interest,Categorica,object,0,0.0,5,3.62,NaN,NaN,NaN,variety,"variety (50), technology (36), none (30)"
9,religion,Categorica,object,0,0.0,5,3.62,NaN,NaN,NaN,Catholic,"Catholic (99), none (30), Christian (7)"



=== PERFIL: usercuisine_csv ===


,Variable,Tipo_inferido,Dtype,Nulos,%Nulos,Unicos,%Cardinalidad,Min,Max,Media,Moda,Top_3
0,userID,Texto,object,0,0.0,138,41.82,None,None,None,U1135,"U1135 (103), U1108 (18), U1101 (15)"
1,Rcuisine,Texto,object,0,0.0,103,31.21,None,None,None,Mexican,"Mexican (97), American (11), Cafeteria (9)"



=== PERFIL: userpayment_csv ===


...

,Variable,Tipo_inferido,Dtype,Nulos,%Nulos,Unicos,%Cardinalidad,Min,Max,Media,Moda,Top_3
0,placeID,Numerica,int64,0,0.0,769,83.95,132001.0,135110.0,132897.174672,None,None
1,Rcuisine,Categorica,object,0,0.0,59,6.44,NaN,NaN,NaN,Mexican,"Mexican (239), International (62), American (59)"



=== PERFIL: payment_methods_csv ===


,Variable,Tipo_inferido,Dtype,Nulos,%Nulos,Unicos,%Cardinalidad,Min,Max,Media,Moda,Top_3
0,placeID,Numerica,int64,0,0.0,615,46.80,132002.0,135110.0,133218.739726,None,None
1,Rpayment,Categorica,object,0,0.0,12,0.91,NaN,NaN,NaN,cash,"cash (500), MasterCard-Eurocard (194), VISA (172)"



=== PERFIL: hours_csv ===


,Variable,Tipo_inferido,Dtype,Nulos,%Nulos,Unicos,%Cardinalidad,Min,Max,Media,Moda,Top_3
0,placeID,Numerica,int64,0,0.0,694,29.67,132012.0,135111.0,133082.311672,None,None
1,hours,Categorica,object,0,0.0,273,11.67,NaN,NaN,NaN,00:00-23:30;,"00:00-23:30; (681), 00:00-00:00; (100), 17:00-..."
2,days,Categorica,object,0,0.0,3,0.13,NaN,NaN,NaN,Mon;Tue;Wed;Thu;Fri;,"Mon;Tue;Wed;Thu;Fri; (793), Sat; (783), Sun; (..."


In [127]:
ped.reporte_relacional(dfs_csv, ratings_key="ratings_csv")

,df,key,rows,unique_keys,min_per_key,max_per_key,mean_per_key,keys_in_universe,pct_keys_in_universe
0,users_csv,userID,138,138,1,1,1.000000,138,1.000000
1,usercuisine_csv,userID,330,138,1,103,2.391304,138,1.000000
2,userpayment_csv,userID,177,133,1,4,1.330827,133,1.000000
3,ratings_csv,userID,1161,138,3,18,8.413043,138,1.000000
4,ratings_csv,placeID,1161,130,3,36,8.930769,130,1.000000
5,restaurants_csv,placeID,130,130,1,1,1.000000,130,1.000000
6,parking_csv,placeID,702,675,1,3,1.040000,130,0.192593
7,cuisine_csv,placeID,916,769,1,9,1.191157,95,0.123537
8,payment_methods_csv,placeID,1314,615,1,8,2.136585,114,0.185366
9,hours_csv,placeID,2339,694,2,12,3.370317,128,0.184438


## Unidad muestral y modelo relacional

A partir del **reporte relacional** (`reporte_relacional`) se identificó que la tabla `ratings_csv` funciona como **tabla de hechos**, ya que contiene directamente las variables de calificación (`rating`, `food_rating`, `service_rating`) y conecta a usuarios y restaurantes mediante las llaves `userID` y `placeID`.

### Evidencia y justificación

- **Tabla de hechos:** `ratings_csv` (1161 filas).
- **Llaves presentes:** `userID` (138 únicos) y `placeID` (130 únicos).
- **Unidad muestral definida:** **1 fila = 1 evaluación** de un usuario (`userID`) a un restaurante (`placeID`).
- **Llave lógica de la evaluación:** (`userID`, `placeID`), y se verificó que **no existen duplicados** para este par, por lo que cada registro representa una evaluación única.

En consecuencia, la base principal para análisis y modelado se construye **partiendo de `ratings_csv`**, preservando su número de filas y, por lo tanto, su unidad muestral.

### Dimensiones analíticas para enriquecer la base

Con el mismo reporte relacional se observó que:
- `users_csv` es **1 a 1** con `userID` (mean_per_key = 1), por lo que puede integrarse directamente a nivel usuario.
- `restaurants_csv` es **1 a 1** con `placeID` (mean_per_key = 1), por lo que puede integrarse directamente a nivel restaurante.
- Otras tablas (`usercuisine_csv`, `userpayment_csv`, `cuisine_csv`, `payment_methods_csv`, `hours_csv`, `parking_csv`) presentan relaciones **1 a muchos** respecto a `userID` o `placeID`, por lo que se transforman previamente mediante **agregaciones** (conteos y listas de categorías) para mantener una sola fila por llave.

Para mantener el notebook legible y robustecer la paquetería, se implementaron funciones que construyen:
- `dim_user` (**usuarios.csv**): tabla analítica a nivel `userID` (1 fila por usuario), integrando `users_csv` y agregados por `userID`.
- `dim_place` (**restaurantes.csv**): tabla analítica a nivel `placeID` (1 fila por restaurante), integrando `restaurants_csv` y agregados por `placeID`.

Finalmente, la **base de modelado** se obtiene realizando `LEFT JOIN` desde `ratings_csv` hacia `dim_user` y `dim_place`, asegurando que el número de filas permanezca en 1161 y que la unidad muestral continúe siendo **la evaluación usuario–restaurante**.


In [ ]:
dim_user = an.construir_dim_usuario(dfs_csv)
dim_place = an.construir_dim_restaurante(dfs_csv)
an.exportar_tablas_analiticas(dim_user, dim_place, out_dir="./data")

base = an.construir_base_modelado(dfs_csv, dim_user, dim_place)

ratings rows: 1161
base rows: 1161


In [145]:
usercuisine = dfs_csv["usercuisine_csv"] 
users = dfs_csv["users_csv"] 
userpayment = dfs_csv["userpayment_csv"] 
ratings = dfs_csv["ratings_csv"] 
parking = dfs_csv["parking_csv"] 
restaurants = dfs_csv["restaurants_csv"] 
cuisine = dfs_csv["cuisine_csv"] 
payment_methods = dfs_csv["payment_methods_csv"] 
hours = dfs_csv["hours_csv"]

,userID,Rcuisine
0,U1001,American
1,U1002,Mexican
2,U1003,Mexican
3,U1004,Bakery
4,U1004,Breakfast-Brunch
...,...,...
325,U1135,Southern
326,U1135,Dessert-Ice_Cream
327,U1136,Mexican
328,U1137,Mexican


In [148]:
base.to_csv("base.csv", index=False)